In [33]:
from IPython.display import display, Image
import cv2 as cv
from PIL import Image

#august
import os
import pandas as pd
from numpy import asarray

In [6]:
#display(Image(filename='brad.jpg'))
#display(Image(filename='kim.jpg'))
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade = cv.CascadeClassifier("haarcascade_eye.xml")

In [58]:
def coords_for_crop(img_address):

    img = cv.imread(img_address)
    im = Image.open(img_address) 

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)

    xUpper, yUpper, xLower, yLower = 0,0,0,0
    
    

    for (x, y, w, h) in faces:
        #print(w,h,type(w),type(h))
        if (w>600 and h> 600):
            return x,y,x+w,y+h
        
        
        #cv.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        #roi_gray = gray[y : y + h, x : x + h]
        #roi_color = img[y : y + h, x : x + h]
        # Otsi silmi vaid leitud nägude aladelt (kollased ristkülikud)
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #for (ex, ey, ew, eh) in eyes:
            #cv.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 255), 2)


    # Otsi silmi kogu fotolt (rohelised ristkülikud)
    #eyes = eye_cascade.detectMultiScale(gray)
    #for (ex, ey, ew, eh) in eyes:
        #cv.rectangle(img, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

    #im1 = im.crop((xUpper, yUpper, xLower, yLower))
    #im1.show()
    #cv.waitKey(0)
    #cv.destroyAllWindows()

In [27]:
# Pildi salvestamiseks
#cv.imwrite('jott_koopia.png', img)

In [64]:
import os
import pandas as pd
from PIL import Image
from numpy import asarray

# Defineerime CSV faili tee ja piltide peamise kausta tee
csv_file_path = './ratings.csv'
images_folder_path = './people'

# Laadime CSV faili
df = pd.read_csv(csv_file_path, header=None)

# Loome tühja DataFrame, kuhu salvestame ümberstruktureeritud andmed
columns = ['usaldusväärsus', 'ilu', 'intelligents', "pilt"]
andmestik_df = pd.DataFrame(columns=columns)

# Itereerime läbi CSV andmete ja lisame need ümberstruktureeritult uude DataFrame'i
for index, row in df.iterrows():
    # Määrame kausta nime vastavalt rea indeksile
    folder_name = f'id{index + 1}'
    folder_path = os.path.join(images_folder_path, folder_name)
    
    # Lisame andmed vastavalt kaustas olevatele piltidele
    for i in range(0, len(row)-3, 3):
        image_name = f'Allalaaditud fail ({i//3 + 1}).jpeg'  # Oletame, et pildid on nimedega img1.jpg, img2.jpg, img3.jpg
        image_path = os.path.join(folder_path, image_name)
        
        # load the image and convert into
        # numpy array
        img = Image.open(image_path)
        
        #crop image so it will only contain the face of the person
        img = img.crop(coords_for_crop(image_path))
        
        #resize image to 600x600 
        img = img.resize((600,600))
        
        ##
        #img.show()
        ##
        
        # asarray() class is used to convert
        # PIL images into NumPy arrays
        image_array = asarray(img)
        
        
        image_data = pd.DataFrame({
            'usaldusväärsus': [row[i]],
            'ilu': [row[i+1]],
            'intelligents': [row[i+2]],
            'pilt': [image_array]  
        })
        andmestik_df = pd.concat([andmestik_df,image_data], ignore_index=True)


# Kui soovite tulemused salvestada uude CSV faili
andmestik_df.to_csv('reshaped_file.csv', index=True)
# Vaatame ümberstruktureeritud andmeid
print(andmestik_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   usaldusväärsus  180 non-null    float64
 1   ilu             180 non-null    float64
 2   intelligents    180 non-null    float64
 3   pilt            210 non-null    object 
dtypes: float64(3), object(1)
memory usage: 6.7+ KB
None
